Link Dataset Compartilhado

https://drive.google.com/file/d/1G-pr-ltNCUq_WYMrgWRZIcdlRANJndy0/view?usp=drive_link

https://drive.google.com/file/d/1G-pr-ltNCUq_WYMrgWRZIcdlRANJndy0/view?usp=sharing

# Instalação de Bibliotecas


In [ ]:
# Instalação de Biblioteca

!pip install torch transformers seqeval pytorch-crf


In [ ]:
# Instalação de Biblioteca para Alibaba QWEN3 modelo arquitetura LLM

!pip install transformers torch seqeval torchcrf accelerate


In [ ]:
# Redução do estouro de memória durante no treinamento, evitando OOM

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [ ]:
# Importar Bibliotecas para Leitura do JSON

import json
import os
from typing import List, Dict, Tuple

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast

In [ ]:
# Instalar GitHub LFS

!git lfs install

In [ ]:
# Baixar arquivo compactado clone modelo treinado do repositório GitHub

!git clone https://github.com/MilitinoAgassi/LLM-NLP-NER-TCM-Tradicional-Chinese-Medicine.git

In [ ]:
# Descompactar arquivo zip

# Criar diretório do destino
!mkdir -p /content/outputs_qwen_ner

# Descompactar arquivos para pasta do destino

!unzip LLM-NLP-NER-TCM-Tradicional-Chinese-Medicine/outputs_qwen_ner.zip -d /content/outputs_qwen_ner

In [ ]:
# Verificar os arquivos descompactados

!ls -lh outputs_qwen_ner

# Leitura do Dataset

## Leitura Database Train (Treino)

In [ ]:
# Leitura Database Train.json

with open ("train.json" , "r" , encoding="utf-8") as File:
  Train_data = json.load(File)

print(Train_data)

In [ ]:
# Visualizar Database Train

for item in Train_data:
  print(item)

In [ ]:
# Imprime as chaves do primeiro elemento Train_data

print(Train_data[0].keys())

In [ ]:
# Imprime a chave labels do Train_data

for item in Train_data:
  Labels_Train_data = item["labels"]
  print(Labels_Train_data)

In [ ]:
# Tamanho Quantidade no Train_data

print(len(Train_data))

In [ ]:
# Tamanho Quantidade Número de Keys no Train_data

print(len(Train_data[0]))

## Leitura Database Dev (Validação)

In [ ]:
# Leitura Database Dev.json

with open ("dev.json" , "r" , encoding="utf-8") as File:
  Dev_data = json.load(File)

print(Dev_data)

In [ ]:
# Visualizar Database Dev

for item in Dev_data:
  print(item)

In [ ]:
# Imprime as chaves do primeiro elemento Dev_data

print(Dev_data[0].keys())

In [ ]:
# Imprime a chave labels do Dev_data

for item in Dev_data:
  Labels_Dev_data = item["labels"]
  print(Labels_Dev_data)

In [ ]:
# Tamanho Quantidade no Dev_data

print(len(Dev_data))

In [ ]:
# Tamanho Quantidade Número de Keys no Dev_data

print(len(Dev_data[0]))


## Leitura Database Test (Teste)

In [ ]:
# Leitura Database Test.json

with open ("test.json" , "r" , encoding="utf-8") as File:
  Test_data = json.load(File)

print(Test_data)

In [ ]:
for item in Test_data:
  print(item)

In [ ]:
# Imprime as chaves do primeiro elemento Test_data

print(Test_data[0].keys())

In [ ]:
# Tamanho Quantidade no Test_data

print(len(Test_data))

In [ ]:
# Tamanho Quantidade Número de Keys no Test_data

print(len(Test_data[0]))

## Leitura Database CRF_ent2id (Entidades Principais)

In [ ]:
# Leitura Database CRF_ent2id.json

with open ("crf_ent2id.json" , "r" , encoding="utf-8") as File:
  CRF_ent2id_data = json.load(File)

print(CRF_ent2id_data)

In [ ]:
# Visualização das Entidades

for item in CRF_ent2id_data:
  print(item)

In [ ]:
# Tamanho Quantidade no CRF_ent2id_data

print(len(CRF_ent2id_data))

## Verificação Dados Duplicados em Train/Dev/Test

In [ ]:
# Versão Avançada

import json

def load_ids(path):
    """Leitura JSON Retornar Todas Amostras id """
    data = json.load(open(path, "r", encoding="utf-8"))
    return set([d["id"] for d in data])

def check_splits(train_path, dev_path, test_path):
    train_ids = load_ids(train_path)
    dev_ids = load_ids(dev_path)
    test_ids = load_ids(test_path)

    print("=== Tamanho Database ===")
    print(f"Train: {len(train_ids)} Dados")
    print(f"Dev:   {len(dev_ids)} Dados")
    print(f"Test:  {len(test_ids)} Dados\n")

    # Verificação Cruzada
    inter_train_dev = train_ids & dev_ids
    inter_train_test = train_ids & test_ids
    inter_dev_test = dev_ids & test_ids

    print("=== Verificação Cruzada ===")
    print("Train ∩ Dev:", inter_train_dev)
    print("Train ∩ Test:", inter_train_test)
    print("Dev ∩ Test:", inter_dev_test)

    # Alerta
    if inter_train_dev or inter_train_test or inter_dev_test:
        print("\n⚠️ Alerta：Amostras Duplicadas！")
        if inter_train_dev:
            print(f"  - Train e Dev Duplicadas {len(inter_train_dev)} Dados")
        if inter_train_test:
            print(f"  - Train e Test Duplicadas {len(inter_train_test)} Dados")
        if inter_dev_test:
            print(f"  - Dev e Test Duplicadas {len(inter_dev_test)} Dados")
    else:
        print("\n✅ Parabéns！Train / Dev / Test Três Databese Independente，Não há Duplicidade。")

if __name__ == "__main__":
    # check_splits("Train_data.json", "Dev_data.json", "Test_data.json")
    check_splits("train.json", "dev.json", "test.json")

# Transformação Limpeza dos Dados / Geração de Embeddings / SPAN BIOS

In [ ]:
# Correção Estrutura Formato Label Entidade do CRF2ID

import json

# Load the current data from crf_ent2id.json
with open("crf_ent2id.json", "r", encoding="utf-8") as f:
    crf_data = json.load(f)

# Extract only the label names (the keys of the dictionary)
label_names = list(crf_data.keys())

# Define the corrected filename
corrected_label_file = "crf_ent2id_corrected.txt"

# Write the corrected label names to a new file, one per line
with open(corrected_label_file, "w", encoding="utf-8") as f:
    for label in label_names:
        f.write(label + "\n")

print(f"Corrected label file saved as: {corrected_label_file}")

# You can now use 'crf_ent2id_corrected.txt' in your conversion script
# For example, you can modify the convert_split function calls to use this new file:
# convert_split(..., label_file=corrected_label_file, ...)

In [ ]:
# Visualizar Label ID CRF2ID Corrigida

import json

def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

label2id, id2label = load_label_set("crf_ent2id_corrected.txt")

print("Label ID to Name Mapping (using crf_ent2id_corrected.txt):")
for id, name in id2label.items():
    print(f"{id}: {name}")

In [ ]:
# Transformação Conversão Alinhamento Normalização Database com SPAN BIOS

# -*- coding: utf-8 -*-
"""
convert_bioes_final.py
 span (start, end, type) alignment BERT tokens，output BIOES Label Sequence。
"""

import json
import re
from typing import List, Dict, Tuple
from transformers import BertTokenizerFast

# ========== LIMPEZA ==========
def clean_text(text: str) -> str:
    if text is None:
        return ""
    text = text.replace("<br/>", "").replace("<br>", "")
    text = text.replace("\u3000", "")
    text = re.sub(r"[\u200b\u200c\u200d\u2060]", "", text)
    text = re.sub(r"[ \t]+", " ", text)
    return text.strip()

# ========== LABEL ==========
def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

# ========== LEIRUTA ==========
def load_json(path: str) -> List[Dict]:
    with open(path, "r", encoding="utf-8") as f:
        content = f.read().strip()
    if not content:
        return []
    if content.startswith("["):
        return json.loads(content)
    return [json.loads(line) for line in content.splitlines() if line.strip()]

# ========== NOMALIZAÇÃO SPAN ==========
def normalize_spans(label_items: List[List], end_inclusive: bool) -> List[Tuple[int, int, str]]:
    spans = []
    for it in label_items or []:
        if len(it) >= 4:
            typ = str(it[1])
            start = int(it[2])
            end = int(it[3])
            if end_inclusive:
                end = end + 1
            spans.append((start, end, typ))
    return spans

# ========== span → BIOES ==========
def spans_to_bioes_for_offsets(text: str, spans: List[Tuple[int, int, str]],
                               offset_mapping: List[Tuple[int, int]],
                               label2id: Dict[str, int]) -> List[int]:
    labels_ids = [label2id["O"]] * len(offset_mapping)

    def token_indices_for_span(start: int, end: int) -> List[int]:
        idxs = []
        for ti, (s, e) in enumerate(offset_mapping):
            if s == e:
                continue
            if max(0, min(e, end) - max(s, start)) > 0:
                idxs.append(ti)
        return idxs

    for (start_char, end_char, etype) in spans:
        idxs = token_indices_for_span(start_char, end_char)
        if not idxs:
            continue
        if len(idxs) == 1:
            labels_ids[idxs[0]] = label2id.get(f"S-{etype}", label2id["O"])
        else:
            labels_ids[idxs[0]] = label2id.get(f"B-{etype}", label2id["O"])
            for k in idxs[1:-1]:
                labels_ids[k] = label2id.get(f"I-{etype}", label2id["O"])
            labels_ids[idxs[-1]] = label2id.get(f"E-{etype}", label2id["O"])
    return labels_ids

# ========== CONVERSÃO ==========
def convert_split(input_json: str, label_file: str, model_name: str,
                  output_path: str, max_len: int = 512,
                  end_inclusive: bool = False, apply_cleaning: bool = True,
                  split_by_char: bool = True):
    label2id, _ = load_label_set(label_file)
    tok = BertTokenizerFast.from_pretrained(model_name)
    data = load_json(input_json)

    out_samples = []
    for ex in data:
        raw_text = ex.get("text", "")
        text = clean_text(raw_text) if apply_cleaning else raw_text

        if split_by_char:
            enc = tok(list(text), is_split_into_words=True,
                      truncation=True, max_length=max_len,
                      return_offsets_mapping=True)
        else:
            enc = tok(text, truncation=True, max_length=max_len,
                      return_offsets_mapping=True)

        spans = normalize_spans(ex.get("labels", []), end_inclusive=end_inclusive)
        labels_ids = spans_to_bioes_for_offsets(text, spans, enc["offset_mapping"], label2id)

        out_samples.append({
            "id": ex.get("id"),
            "input_ids": enc["input_ids"],
            "attention_mask": enc["attention_mask"],
            "labels": labels_ids
        })

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(out_samples, f, ensure_ascii=False, indent=2)
    print(f"[OK] Converted {len(out_samples)} samples → {output_path}")

# Call the conversion functions directly
convert_split(
    input_json="train.json",
    label_file="crf_ent2id_corrected.txt",
    model_name="hfl/chinese-bert-wwm-ext",
    output_path="converted_train.json",
    max_len=512,
    end_inclusive=False,
    apply_cleaning=True,
    split_by_char=False,
)

convert_split(
    input_json="dev.json",
    label_file="crf_ent2id_corrected.txt",
    model_name="hfl/chinese-bert-wwm-ext",
    output_path="converted_dev.json",
    max_len=512,
    end_inclusive=False,
    apply_cleaning=True,
    split_by_char=False,
)



In [ ]:
# Distribuição do Label Type

from collections import Counter
import json

# Load the converted training data
with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

# Load id2label mapping from the corrected file
def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

label2id, id2label = load_label_set("crf_ent2id_corrected.txt")

# Extract all label IDs from the training data
all_train_labels = []
for entry in converted_train_data:
    all_train_labels.extend(entry["labels"])

# Count the occurrences of each label ID
label_id_counts = Counter(all_train_labels)

# Convert label IDs to names and print the counts
print("Distribution of Label Types in converted_train.json (using corrected labels):")
for label_id, count in label_id_counts.items():
    label_name = id2label.get(label_id, f"Unknown ID: {label_id}")
    print(f"{label_name}: {count}")

In [ ]:
# Visualizar Dataset Train convertido para Alimentar Input Treinamento do Modelo

import json

with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

# Print the first 5 entries
for i, entry in enumerate(converted_train_data[:5]):
    print(entry)
    if i == 4:
        break

In [ ]:
# Visualizar Estrutura Sequência para Input_ID e Label para Treinamento do Modelo

import json

with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

print("Structure of input_ids and labels:")
for i, entry in enumerate(converted_train_data[:3]): # Displaying first 3 examples
    print(f"--- Example {i+1} ---")
    print(f"  input_ids: {entry['input_ids']}")
    print(f"  Length of input_ids: {len(entry['input_ids'])}")
    print(f"  labels: {entry['labels']}")
    print(f"  Length of labels: {len(entry['labels'])}")
    print("-" * 20)

In [ ]:
# Plot Distribuião Comprimento da Sequência dos Dados Convertidos para Treino

import json
import matplotlib.pyplot as plt

# Load the converted data
with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

with open("converted_dev.json", "r", encoding="utf-8") as f:
    converted_dev_data = json.load(f)

# Get the lengths of the labels sequences
train_label_lengths = [len(entry["labels"]) for entry in converted_train_data]
dev_label_lengths = [len(entry["labels"]) for entry in converted_dev_data]

# Combine lengths for visualization
all_label_lengths = train_label_lengths + dev_label_lengths

# Create a histogram
plt.figure(figsize=(10, 6))
plt.hist(all_label_lengths, bins=50, edgecolor='black')
plt.xlabel("Sequence Length (Number of Tokens)")
plt.ylabel("Frequency")
plt.title("Distribution of Sequence Lengths in Converted Data")
plt.grid(axis='y', alpha=0.75)
plt.show()

print(f"Minimum sequence length: {min(all_label_lengths)}")
print(f"Maximum sequence length: {max(all_label_lengths)}")
print(f"Average sequence length: {sum(all_label_lengths) / len(all_label_lengths):.2f}")

In [ ]:
# Plot Distribuição Label Type Dos Dados Convertidos para Treino

from collections import Counter
import json
import matplotlib.pyplot as plt

# Load the converted training data
with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

# Load id2label mapping from the corrected file
def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

label2id, id2label = load_label_set("crf_ent2id_corrected.txt")

# Extract all label IDs from the training data
all_train_labels = []
for entry in converted_train_data:
    all_train_labels.extend(entry["labels"])

# Count the occurrences of each label ID
label_id_counts = Counter(all_train_labels)

# Convert label IDs to names and prepare for plotting
label_names = [id2label.get(label_id, f"Unknown ID: {label_id}") for label_id in label_id_counts.keys()]
counts = list(label_id_counts.values())

# Sort labels by count for better visualization
sorted_labels_and_counts = sorted(zip(label_names, counts), key=lambda item: item[1], reverse=True)
sorted_label_names = [item[0] for item in sorted_labels_and_counts]
sorted_counts = [item[1] for item in sorted_labels_and_counts]

# Create the bar plot
plt.figure(figsize=(15, 7)) # Increased figure size for better readability
plt.bar(sorted_label_names, sorted_counts)
plt.xlabel("Label Type")
plt.ylabel("Frequency")
plt.title("Distribution of Label Types in Converted Training Data (Corrected Labels)")
plt.xticks(rotation=90) # Rotate labels to prevent overlap
plt.tight_layout() # Adjust layout to prevent labels from being cut off
plt.show()

In [ ]:
# Plot Distribuição Label Type Dos Dados Convertidos para Treino (Exclusão Label Type "O" = Others)


from collections import Counter
import json
import matplotlib.pyplot as plt

# Load the converted training data
with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

# Load id2label mapping from the corrected file
def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

label2id, id2label = load_label_set("crf_ent2id_corrected.txt")

# Extract all label IDs from the training data
all_train_labels = []
for entry in converted_train_data:
    all_train_labels.extend(entry["labels"])

# Count the occurrences of each label ID
label_id_counts = Counter(all_train_labels)

# Convert label IDs to names and prepare for plotting, excluding the 'O' label
label_names = []
counts = []
for label_id, count in label_id_counts.items():
    label_name = id2label.get(label_id, f"Unknown ID: {label_id}")
    if label_name != 'O': # Exclude the 'O' label
        label_names.append(label_name)
        counts.append(count)

# Sort labels by count for better visualization
sorted_labels_and_counts = sorted(zip(label_names, counts), key=lambda item: item[1], reverse=True)
sorted_label_names = [item[0] for item in sorted_labels_and_counts]
sorted_counts = [item[1] for item in sorted_labels_and_counts]

# Create the bar plot
plt.figure(figsize=(15, 7)) # Increased figure size for better readability
plt.bar(sorted_label_names, sorted_counts)
plt.xlabel("Label Type (Excluding 'O')")
plt.ylabel("Frequency")
plt.title("Distribution of Label Types in Converted Training Data (Excluding 'O')")
plt.xticks(rotation=90) # Rotate labels to prevent overlap
plt.tight_layout() # Adjust layout to prevent labels from being cut off
plt.show()

print("Exact frequency of label types (excluding 'O'):")
for name, count in sorted_labels_and_counts:
    print(f"{name}: {count}")

In [ ]:
# Comparativo Exemplo Amostra Texto Original e Texto Convertido

import json

# Load original training data
with open("train.json", "r", encoding="utf-8") as f:
    original_train_data = json.load(f)

# Load converted training data
with open("converted_train.json", "r", encoding="utf-8") as f:
    converted_train_data = json.load(f)

# Load id2label mapping from the corrected file
def load_label_set(label_file: str) -> Tuple[Dict[str, int], Dict[int, str]]:
    labels = []
    with open(label_file, "r", encoding="utf-8") as f:
        for line in f:
            t = line.strip()
            if t:
                labels.append(t)
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}
    return label2id, id2label

label2id, id2label = load_label_set("crf_ent2id_corrected.txt")


# Display the first few examples
for i in range(3): # Displaying first 3 examples
    original_entry = original_train_data[i]
    converted_entry = converted_train_data[i]

    print(f"--- Example {i+1} ---")
    print("Original Text:")
    print(original_entry['text'])
    print("\nConverted Data:")
    print(f"  ID: {converted_entry['id']}")
    print(f"  Input IDs: {converted_entry['input_ids']}")
    print(f"  Attention Mask: {converted_entry['attention_mask']}")
    print(f"  Labels (IDs): {converted_entry['labels']}")

    # Convert label IDs to names for better readability
    label_names = [id2label[label_id] for label_id in converted_entry['labels']]
    print(f"  Labels (Names): {label_names}")

    print("-" * 20)

In [ ]:
# Visualizar Alinhamento do Texto Após da Limpeza

# -*- coding: utf-8 -*-


import json
import re
from transformers import BertTokenizerFast

def clean_text(text: str) -> str:
    """Limpeza HTML espaçõ"""
    text = re.sub(r"<br\s*/?>", "", text)
    text = re.sub(r"<.*?>", "", text)
    text = text.replace("\u3000", " ")
    return text

def load_sample(path, sample_id=None):
    data = json.load(open(path, "r", encoding="utf-8"))

    # Listar primeiros 50 id
    ids = [d["id"] for d in data]
    print("所有可用的 id:", ids[:50])
    print("\n\n")

    if sample_id is None:
        return data[0]
    for d in data:
        if str(d["id"]) == str(sample_id):
            return d
    raise ValueError(f"找不到 id={sample_id} 的樣本")

def spans_to_bioes(text, spans):
    labels = ["O"] * len(text)
    for s in spans:
        if len(s) < 4: continue
        typ = s[1]
        st, ed = int(s[2]), int(s[3])
        if ed <= st or st < 0 or ed > len(text): continue
        if ed - st == 1:
            labels[st] = f"S-{typ}"
        else:
            labels[st] = f"B-{typ}"
            for i in range(st+1, ed-1):
                labels[i] = f"I-{typ}"
            labels[ed-1] = f"E-{typ}"
    return labels

def debug_one(sample, tokenizer):
    # Before tokenizer fazer limpeza
    raw_text = sample["text"]
    text = clean_text(raw_text)

    spans = sample.get("labels", [])
    print("=== 原文（清洗後）===")
    print(text)
    print("\n=== Gold spans ===")
    for s in spans:
        print(s)

    gold_bioes = spans_to_bioes(text, spans)

    # Correct tokenizer mapping
    enc = tokenizer(text, return_offsets_mapping=True,
                    truncation=True, max_length=512)
    tokens = tokenizer.convert_ids_to_tokens(enc["input_ids"])
    offsets = enc["offset_mapping"]

    print("\n=== Token Check Alinhamento ===")
    for i, (tok, (st, ed)) in enumerate(zip(tokens, offsets)):
        if st == ed == 0:
            label = "O"
        elif st < len(gold_bioes):
            label = gold_bioes[st]
        else:
            label = "O"
        frag = text[st:ed] if st < ed else ""
        print(f"{i:3d} | token={tok:8s} | offset=({st:3d},{ed:3d}) | text='{frag}' | label={label}")

# Notebook main
def main(data_json="Train_data.json", sample_id=None, model_name="hfl/chinese-bert-wwm-ext"):
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
    sample = load_sample(data_json, sample_id)
    debug_one(sample, tokenizer)

# Parâmetro
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_json", type=str, default="train.json")
    parser.add_argument("--sample_id", type=str, default=923)
    parser.add_argument("--model_name", type=str, default="hfl/chinese-bert-wwm-ext")
    args, unknown = parser.parse_known_args()
    main(data_json=args.data_json, sample_id=args.sample_id, model_name=args.model_name)

# Treinamento do Modelo Alibaba QWEN3-0.6B LLM

## Test:7 Batch Size:2 Grad Accum Steps:4 Warmup Ratio:0.1 Dropout:0.1 Epochs:20

In [ ]:
# -*- coding: utf-8 -*-
"""
train_ner_final.py
Read convert_bioes_final.py Output JSON（input_ids/attention_mask/labels），
Qwen(+CRF True/False) Training NER，：
- Gradient Accumlative Steps (grad_accum_steps)
- AMP Automatic Mixed Precision (torch.cuda.amp.autocast + GradScaler)
- Scheduler Based on parameter update steps
- Training Monitoring: Loss/F1 curves
- Best Model / Training Log CSV / Training Curves PNG

- Train F1 + Validation F1
- Plot Curves (Loss / Train F1 / Val F1)
- Download outputs_qwen_ner/
"""

import os
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, precision_score, recall_score, classification_report
from torchcrf import CRF
from torch.optim import AdamW
import matplotlib.pyplot as plt
import pandas as pd


# OOM
#import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# ========== DATA ==========
class NERJsonDataset(Dataset):
    def __init__(self, path: str):
        with open(path, "r", encoding="utf-8") as f:
            self.items = json.load(f)

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        ex = self.items[idx]
        return {
            "input_ids": torch.tensor(ex["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(ex["attention_mask"], dtype=torch.long),
            "labels": torch.tensor(ex["labels"], dtype=torch.long),
        }

def pad_collate(batch, pad_token_id=0, pad_label_id=0):
    max_len = max(len(x["input_ids"]) for x in batch)
    out = {}
    for k in ["input_ids", "attention_mask", "labels"]:
        seqs = []
        for x in batch:
            t = x[k]
            pad_id = pad_token_id if k != "labels" else pad_label_id
            if len(t) < max_len:
                t = torch.cat([t, torch.full((max_len - len(t),), pad_id, dtype=t.dtype)])
            seqs.append(t)
        out[k] = torch.stack(seqs)
    return out

# ========== MODEL ==========
class QwenForTokenClassification(nn.Module):
    def __init__(self, model_name: str, num_labels: int, use_crf: bool = False, dropout: float = 0.1):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.use_crf = use_crf
        if use_crf:
            self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, labels=None):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        seq = self.dropout(out.last_hidden_state)
        logits = self.classifier(seq)

        loss = None
        if labels is not None:
            if self.use_crf:
                loss = -self.crf(logits, labels, mask=attention_mask.bool(), reduction='mean')
            else:
                active = attention_mask.view(-1) == 1
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(
                    logits.view(-1, logits.size(-1))[active],
                    labels.view(-1)[active]
                )

        if self.use_crf:
            pred = self.crf.decode(logits, mask=attention_mask.bool())
            max_len = logits.size(1)
            pred = torch.tensor([p + [0] * (max_len - len(p)) for p in pred], dtype=torch.long, device=logits.device)
        else:
            pred = torch.argmax(logits, dim=-1)

        return {"loss": loss, "pred_ids": pred, "logits": logits}

# ========== ALIGN EVALUATE ==========
def align_eval(pred_ids, labels, attention_mask, id2label):
    preds, trues = [], []
    bs, L = pred_ids.size()
    for i in range(bs):
        p_seq, t_seq = [], []
        for j in range(L):
            if attention_mask[i, j].item() == 1:
                p_seq.append(id2label[pred_ids[i, j].item()])
                t_seq.append(id2label[labels[i, j].item()])
        preds.append(p_seq)
        trues.append(t_seq)
    return preds, trues

# ========== MAIN ==========
def main(argv=None):
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("--converted_train", type=str, default="converted_train.json")
    parser.add_argument("--converted_dev", type=str, default="converted_dev.json")
    parser.add_argument("--labels_file", type=str, default="crf_ent2id_corrected.txt")
    parser.add_argument("--model_name", type=str, default="Qwen/Qwen3-0.6B")

    parser.add_argument("--output_dir", type=str, default="outputs_qwen_ner")
    parser.add_argument("--batch_size", type=int, default=2)
    parser.add_argument("--epochs", type=int, default=20)
    parser.add_argument("--lr", type=float, default=3e-5)
    parser.add_argument("--warmup_ratio", type=float, default=0.1)
    parser.add_argument("--dropout", type=float, default=0.1)

    parser.add_argument("--use_crf", action="store_true", default=True)
    parser.add_argument("--grad_accum_steps", type=int, default=4, help="Gradient Accumulative Steps（Ex: 4）")
    parser.add_argument("--use_amp", action="store_true", default=True, help="Active AMP Automatic Mixed Precision（CUDA）")
    if argv is None:
        args, _ = parser.parse_known_args()
    else:
        args = parser.parse_args(argv)

    print(f"Model: {args.model_name} | Batch size: {args.batch_size} | "
          f"CRF enabled: {args.use_crf} | GradAccum: {args.grad_accum_steps} | "
          f"Epochs: {args.epochs} | LR: {args.lr} | Warmup ratio: {args.warmup_ratio} | "
          f"AMP: {args.use_amp} | Dropout: {args.dropout}")

    # LABEL
    with open(args.labels_file, "r", encoding="utf-8") as f:
        labels = [l.strip() for l in f if l.strip()]
    if "O" not in labels:
        labels = ["O"] + labels
    label2id = {l: i for i, l in enumerate(labels)}
    id2label = {i: l for l, i in label2id.items()}

    tokenizer = AutoTokenizer.from_pretrained(args.model_name)

    # DATA and DataLoader
    train_ds = NERJsonDataset(args.converted_train)
    dev_ds = NERJsonDataset(args.converted_dev)

    train_loader = DataLoader(
        train_ds,
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=lambda b: pad_collate(
            b,
            pad_token_id=tokenizer.pad_token_id or 0,
            pad_label_id=label2id["O"]
        )
    )
    dev_loader = DataLoader(
        dev_ds,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=lambda b: pad_collate(
            b,
            pad_token_id=tokenizer.pad_token_id or 0,
            pad_label_id=label2id["O"]
        )
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = QwenForTokenClassification(
        args.model_name,
        num_labels=len(label2id),
        use_crf=args.use_crf,
        dropout=args.dropout
    ).to(device)

    optimizer = AdamW(model.parameters(), lr=args.lr)

    # UPDATE BASED PARAMETERS scheduler
    total_update_steps = (len(train_loader) * args.epochs) // max(args.grad_accum_steps, 1)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_update_steps * args.warmup_ratio),
        num_training_steps=total_update_steps
    )

    # AMP
    use_amp = args.use_amp and torch.cuda.is_available()
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    autocast = torch.cuda.amp.autocast

    # ====== MONITORING / RECORD ======
    os.makedirs(args.output_dir, exist_ok=True)
    train_losses, train_f1s, val_f1s = [], [], []
    best_f1 = 0.0

    for epoch in range(1, args.epochs + 1):
        # ===== TRAINING =====
        model.train()
        total_loss = 0.0
        optimizer.zero_grad()

        for step, batch in enumerate(train_loader, 1):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with autocast(enabled=use_amp):
                out = model(input_ids, attention_mask, labels)
                loss = out["loss"]

            # GRADIENT ACCUMULATIVE STEPS
            loss = loss / max(args.grad_accum_steps, 1)
            scaler.scale(loss).backward()
            total_loss += loss.item()

            if step % max(args.grad_accum_steps, 1) == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                optimizer.zero_grad()

        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)

        # ===== EVALUATE =====
        model.eval()
        all_preds_val, all_trues_val = [], []
        with torch.no_grad():
            for batch in dev_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)
                out = model(input_ids, attention_mask, labels=None)
                preds, trues = align_eval(out["pred_ids"], labels, attention_mask, id2label)
                all_preds_val.extend(preds)
                all_trues_val.extend(trues)

        f1_val = f1_score(all_trues_val, all_preds_val)
        p_val = precision_score(all_trues_val, all_preds_val)
        r_val = recall_score(all_trues_val, all_preds_val)
        val_f1s.append(f1_val)

        # ===== TRAIN F1 =====
        all_preds_train, all_trues_train = [], []
        with torch.no_grad():
            for batch in train_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)
                out = model(input_ids, attention_mask, labels=None)
                preds, trues = align_eval(out["pred_ids"], labels, attention_mask, id2label)
                all_preds_train.extend(preds)
                all_trues_train.extend(trues)
        f1_train = f1_score(all_trues_train, all_preds_train)
        train_f1s.append(f1_train)

        print(f"Epoch {epoch} | Loss={avg_loss:.4f} | "
              f"TrainF1={f1_train:.4f} | ValF1={f1_val:.4f} | P={p_val:.4f} | R={r_val:.4f}")
        print(classification_report(all_trues_val, all_preds_val, digits=4))



        # BEST MODEL RECORD（VALIDATION F1）
        if f1_val > best_f1:
            best_f1 = f1_val
            torch.save(model.state_dict(), os.path.join(args.output_dir, "pytorch_model.bin"))
            tokenizer.save_pretrained(args.output_dir)
            with open(os.path.join(args.output_dir, "label2id.json"), "w", encoding="utf-8") as f:
                json.dump(label2id, f, ensure_ascii=False, indent=2)
            print(f"Saved best model to {args.output_dir} (F1={best_f1:.4f})")

    print(f"Training finished. Best F1: {best_f1:.4f}")

    # ===== PLOT Loss/F1 CURVES =====
    epochs_axis = range(1, len(train_losses)+1)
    fig, ax1 = plt.subplots()
    ax1.plot(epochs_axis, train_losses, 'r-o', label='Train Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss', color='r')
    ax1.tick_params(axis='y', labelcolor='r')
    ax2 = ax1.twinx()
    ax2.plot(epochs_axis, val_f1s, 'b-s', label='Validation F1')
    ax2.set_ylabel('F1 Score', color='b')
    ax2.tick_params(axis='y', labelcolor='b')
    fig.suptitle('Training Progress: Loss vs F1')
    ax1.grid(True)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    plt.tight_layout()
    plt.savefig(os.path.join(args.output_dir, "training_curves.png"))
    plt.show()


    print("\n\n")


    # ===== PLOT 3 LINE CURVES =====
    epochs_axis = range(1, len(train_losses)+1)
    plt.figure(figsize=(8,6))
    plt.plot(epochs_axis, train_losses, 'r-o', label='Train Loss')
    plt.plot(epochs_axis, train_f1s, 'g-^', label='Train F1')
    plt.plot(epochs_axis, val_f1s, 'b-s', label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training Progress: Loss vs Train F1 vs Validation F1')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    curve_path = os.path.join(args.output_dir, "training_curves_three.png")
    plt.savefig(curve_path)
    plt.show()
    print(f"Curves image saved to {curve_path}")

    # ===== RECORD LOG CSV =====
    log_df = pd.DataFrame({
        "epoch": list(range(1, len(train_losses)+1)),
        "train_loss": train_losses,
        "val_f1": val_f1s
    })
    log_path = os.path.join(args.output_dir, "training_log.csv")
    log_df.to_csv(log_path, index=False, encoding="utf-8")
    print(f"Training log saved to {log_path}")
    print(f"Curves image saved to {os.path.join(args.output_dir, 'training_curves.png')}")


    # ===== DOWNLOAD =====
    import shutil
    zip_path = f"{args.output_dir}.zip"
    shutil.make_archive(args.output_dir, 'zip', args.output_dir)
    print(f"📦 Finish：{zip_path}")

    # Colab Download Automatic（Not Colab，Fail，Manual）
    try:
        from google.colab import files
        files.download(zip_path)
    except Exception as e:
        print(f"Automatic download skipped（Not Colab or Download Failed）：{e}")
        print(f"Please download manually：{zip_path}")

if __name__ == "__main__":
    main()

# Predição Teste

## Predict Avaliação com Test_Gold ou Test

In [ ]:
# predict_eval_final.py
import os
import json
import datetime
from collections import defaultdict, Counter

import torch
import torch.nn as nn
import pandas as pd
from torchcrf import CRF
from transformers import AutoTokenizer, AutoModel


# ===== MODEL TYPE =====
class QwenForTokenClassification(nn.Module):
    def __init__(self, model_name: str, num_labels: int, use_crf: bool = True, dropout: float = 0.1):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True) if use_crf else None

    def forward(self, input_ids, attention_mask):
        # input_ids: [B, L], attention_mask: [B, L]
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        seq = self.dropout(out.last_hidden_state)           # [B, L, H]
        emissions = self.classifier(seq)                    # [B, L, num_labels]
        if self.crf is None:
            return emissions.argmax(dim=-1)                 # [B, L]
        # CRF decode RETURN list[list[int]]（DIFF SEQUENCE LENGTH）
        pred_paths = self.crf.decode(emissions, mask=attention_mask.bool())
        max_len = emissions.size(1)
        # PADDED WITH O TYPE（id=0）FIX LENGTH
        padded = torch.tensor(
            [p + [0] * (max_len - len(p)) for p in pred_paths],
            dtype=torch.long,
            device=emissions.device
        )                                                   # [B, L]
        return padded


# ===== BIOES → spans DECODE（end WITHOUT） =====
def bioes_to_spans(labels, offsets):
    ents = []
    cur_lab, cur_s, cur_e = None, None, None
    for lab, (s, e) in zip(labels, offsets):
        if lab.startswith("B-"):
            cur_lab = lab[2:]; cur_s = s; cur_e = e
        elif lab.startswith("I-") and cur_lab == lab[2:]:
            cur_e = e
        elif lab.startswith("E-") and cur_lab == lab[2:]:
            cur_e = e
            ents.append((cur_s, cur_e, cur_lab))
            cur_lab, cur_s, cur_e = None, None, None
        elif lab.startswith("S-"):
            ents.append((s, e, lab[2:]))
            cur_lab, cur_s, cur_e = None, None, None
        else:
            # O OR DISCONTINUOUS
            cur_lab, cur_s, cur_e = None, None, None
    return ents


# ===== SINGLE SAMPLE TEXT → spans PREDICT =====
def predict_spans(text, tokenizer, model, id2label, device):
    enc = tokenizer(
        text,
        return_offsets_mapping=True,
        return_tensors="pt",
        truncation=True
    )
    offsets = enc["offset_mapping"][0].tolist()
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        pred_ids = model(input_ids=input_ids, attention_mask=attention_mask)[0].tolist()

    pred_labels = [id2label[i] for i in pred_ids]
    valid_offsets, valid_labels = [], []
    for lab, off in zip(pred_labels, offsets):
        # FILTER SPECIAL token（WHEN offset = (0,0)）
        if off != [0, 0] and off != (0, 0):
            valid_offsets.append(tuple(off))
            valid_labels.append(lab)
    return bioes_to_spans(valid_labels, valid_offsets)


def span_key(s, e, l):
    return f"{s}-{e}-{l}"


# ===== ENTITY EVALUATE =====
def evaluate_entity_level(gold_items, tokenizer, model, id2label, device):
    tp, fp, fn = 0, 0, 0
    per_type = defaultdict(lambda: Counter({"tp": 0, "fp": 0, "fn": 0}))

    for item in gold_items:
        text = item["text"]
        pred_spans = predict_spans(text, tokenizer, model, id2label, device)
        gold_spans = [(ent["start"], ent["end"], ent["label"]) for ent in item["entities"]]

        pred_set = {span_key(*sp) for sp in pred_spans}
        gold_set = {span_key(*sp) for sp in gold_spans}

        tp_set = pred_set & gold_set
        fp_set = pred_set - gold_set
        fn_set = gold_set - pred_set

        tp += len(tp_set)
        fp += len(fp_set)
        fn += len(fn_set)

        for s, e, l in pred_spans:
            if span_key(s, e, l) in tp_set:
                per_type[l]["tp"] += 1
            else:
                per_type[l]["fp"] += 1
        for s, e, l in gold_spans:
            if span_key(s, e, l) in fn_set:
                per_type[l]["fn"] += 1

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    per_type_report = {}
    for t, c in per_type.items():
        p = c["tp"] / (c["tp"] + c["fp"]) if (c["tp"] + c["fp"]) > 0 else 0.0
        r = c["tp"] / (c["tp"] + c["fn"]) if (c["tp"] + c["fn"]) > 0 else 0.0
        f = 2 * p * r / (p + r) if (p + r) > 0 else 0.0
        support = c["tp"] + c["fn"]
        per_type_report[t] = {
            "precision": round(p, 4),
            "recall": round(r, 4),
            "f1": round(f, 4),
            "support": int(support),
        }

    return {
        "micro": {
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "f1": round(f1, 4),
            "tp": int(tp),
            "fp": int(fp),
            "fn": int(fn),
        },
        "per_type": per_type_report,
    }


# ===== NO GOLD STANDARD STATISTICS =====
def predict_and_count(test_items, tokenizer, model, id2label, device):
    counts = Counter()
    all_preds = []
    for item in test_items:
        text = item["text"]
        pred_spans = predict_spans(text, tokenizer, model, id2label, device)
        all_preds.append({
            "id": item.get("id"),
            "text": text,
            "pred_entities": [{"start": s, "end": e, "label": l} for s, e, l in pred_spans],
        })
        for _, _, l in pred_spans:
            counts[l] += 1
    return all_preds, counts


# ===== CSV OUTPUT =====
def save_eval_csv(report, output_dir, timestamp):
    os.makedirs(output_dir, exist_ok=True)
    micro_df = pd.DataFrame([report["micro"]])
    micro_path = os.path.join(output_dir, f"micro_eval_{timestamp}.csv")
    micro_df.to_csv(micro_path, index=False, encoding="utf-8")

    per_df = pd.DataFrame([{"label": lbl, **metrics} for lbl, metrics in report["per_type"].items()])
    per_path = os.path.join(output_dir, f"per_type_eval_{timestamp}.csv")
    per_df.to_csv(per_path, index=False, encoding="utf-8")

    print(f"✅ OUTPUT CSV：{micro_path}")
    print(f"✅ OUTPUT CSV：{per_path}")


def save_stats_csv(counts, output_dir, timestamp):
    os.makedirs(output_dir, exist_ok=True)
    df = pd.DataFrame([{"label": l, "count": c} for l, c in counts.items()])
    path = os.path.join(output_dir, f"prediction_stats_{timestamp}.csv")
    df.to_csv(path, index=False, encoding="utf-8")
    print(f"✅ OUTPUT CSV：{path}")


# ===== MAIN =====
def main():
    # DIRECTORY PATH
    output_dir = "outputs_qwen_ner"
    gold_path = "test_gold.json"
    test_path = "test.json"

    # TIMSTAMP
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # DEVICE
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️ USE DEVICE: {device}")

    # LOAD tokenizer MAP TAG
    tokenizer = AutoTokenizer.from_pretrained(output_dir)
    with open(os.path.join(output_dir, "label2id.json"), "r", encoding="utf-8") as f:
        label2id = json.load(f)
    id2label = {v: k for k, v in label2id.items()}

    # LOAD MODEL
    # model_name training backbone name
    model = QwenForTokenClassification(model_name="Qwen/Qwen3-0.6B", num_labels=len(label2id))
    state = torch.load(os.path.join(output_dir, "pytorch_model.bin"), map_location="cpu")
    model.load_state_dict(state)
    model.to(device)
    model.eval()
    print("✅ Best model loaded.")

    if os.path.exists(gold_path):
        print("🔎 Detected Test_Gold.json → ENTITY EVALUATE")
        gold_items = json.load(open(gold_path, "r", encoding="utf-8"))
        report = evaluate_entity_level(gold_items, tokenizer, model, id2label, device)

        # PRINT micro RESULT
        micro = report["micro"]
        print("=== Entity-level Micro Scores ===")
        print(f"Precision: {micro['precision']}")
        print(f"Recall   : {micro['recall']}")
        print(f"F1       : {micro['f1']}")
        print(f"TP={micro['tp']}  FP={micro['fp']}  FN={micro['fn']}")

        # PRINT per-type REPORT
        print("\n=== Per-type Classification Report ===")
        for t, m in sorted(report["per_type"].items()):
            print(f"{t:25s}  P={m['precision']:.4f}  R={m['recall']:.4f}  F1={m['f1']:.4f}  Support={m['support']}")

        # SAVE CSV
        save_eval_csv(report, output_dir=output_dir, timestamp=timestamp)

    else:
        print("🔎 NOT FOUND Test_Gold.json → USE Test.json DO PREDICTIVE STATISTICS")
        test_items = json.load(open(test_path, "r", encoding="utf-8"))
        preds, counts = predict_and_count(test_items, tokenizer, model, id2label, device)

        # PRINT STATISTICS
        print("=== Prediction Statistics (Counts by label) ===")
        for l, c in sorted(counts.items()):
            print(f"{l:25s}: {c}")

        # SAVE PREDICT JSON
        os.makedirs(output_dir, exist_ok=True)
        pred_path = os.path.join(output_dir, f"Test_Pred_{timestamp}.json")
        with open(pred_path, "w", encoding="utf-8") as f:
            json.dump(preds, f, ensure_ascii=False, indent=2)
        print(f"✅ PREDICTION RESULTS HAVE BEEN OUTPU TO  {pred_path}")

        # SAVE CSV STATISTICS
        save_stats_csv(counts, output_dir=output_dir, timestamp=timestamp)


if __name__ == "__main__":
    main()


## Carregar Modelo Treinado pelo GitHub Repositório

In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
from torchcrf import CRF

output_dir = "/content/outputs_qwen_ner"

# LOAD tokenizer AND label2id
tokenizer = AutoTokenizer.from_pretrained(output_dir)
with open(f"{output_dir}/label2id.json", "r", encoding="utf-8") as f:
    label2id = json.load(f)
id2label = {v:k for k,v in label2id.items()}

# DEFINE CUSTOM MODEL
class QwenForTokenClassification(nn.Module):
    def __init__(self, model_name: str, num_labels: int, use_crf: bool = True, dropout: float = 0.1):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True) if use_crf else None

    def forward(self, input_ids, attention_mask):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        seq = self.dropout(out.last_hidden_state)
        emissions = self.classifier(seq)
        if self.crf:
            paths = self.crf.decode(emissions, mask=attention_mask.bool())
            max_len = emissions.size(1)
            padded = torch.tensor([p + [0]*(max_len - len(p)) for p in paths], dtype=torch.long)
            return padded
        else:
            return emissions.argmax(dim=-1)

# LOAD THE OPTIMAL MODEL WEIGHTS
model = QwenForTokenClassification(model_name="Qwen/Qwen3-0.6B", num_labels=len(label2id))
model.load_state_dict(torch.load(f"{output_dir}/pytorch_model.bin", map_location="cpu"))
model.eval()
print("✅ Best model loaded from GitHub repo")

In [ ]:
# QUICK PREDICT TEST (SINGLE SENTENCE / OUTPUT SPAN)


import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


def bioes_to_spans(labels, offsets):
    ents, cur_lab, cur_s, cur_e = [], None, None, None
    for lab, (s, e) in zip(labels, offsets):
        if lab.startswith("B-"):
            cur_lab, cur_s, cur_e = lab[2:], s, e
        elif lab.startswith("I-") and cur_lab == lab[2:]:
            cur_e = e
        elif lab.startswith("E-") and cur_lab == lab[2:]:
            cur_e = e
            ents.append((cur_s, cur_e, cur_lab))
            cur_lab, cur_s, cur_e = None, None, None
        elif lab.startswith("S-"):
            ents.append((s, e, lab[2:]))
            cur_lab, cur_s, cur_e = None, None, None
        else:
            cur_lab, cur_s, cur_e = None, None, None
    return ents


def predict_spans(text):
    enc = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True)
    offsets = enc["offset_mapping"][0].tolist()
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    with torch.no_grad():
        pred_ids = model(input_ids=input_ids, attention_mask=attention_mask)[0].tolist()

    pred_labels = [id2label[i] for i in pred_ids]
    valid_offsets, valid_labels = [], []
    for lab, off in zip(pred_labels, offsets):
        if off != [0, 0] and off != (0, 0):
            valid_offsets.append(tuple(off))
            valid_labels.append(lab)
    return bioes_to_spans(valid_labels, valid_offsets)

#sample_text = "患者服用某藥物後出現頭痛與噁心，建議與食物同服。"
sample_text = "本品为黑褐色颗粒；气香，味甜。  株洲千金药业股份有限公司  忌生冷辛辣，孕妇禁服。  开水冲服，一次12g，一日2次。  12g*10袋 补益气血，祛瘀生新。用于气血两虚兼血瘀证产后腹痛 动物试验表明，补血益母颗粒能使失血性贫血小鼠RBC、Hb恢复至正常水平;能对抗环磷酰胺损伤骨髓造血系统所致的血细胞减少，能使WBC、HB、RBC明显升高;且对环磷酰胺所致小鼠脾脏萎缩有明显的对抗作用;它对小鼠既具有活血作用又能缩短小鼠的凝血时间;提高小鼠巨噬细胞的吞噬功能和促进小鼠溶血素抗体的形成;促进小鼠腹腔绵羊红细胞的吸收;但对正常大鼠离体子宫平滑肌未显示出作用。  12g*10袋/盒。  1.忌食寒凉、生冷食物。2.感冒时不宜服用。3.平素月经正常，突然出现月经量少，或月经错后，或阴道不规则出血应去医院就诊。4.按照用法用量服用，长期服用应向医师咨询。5.服药二周症状无改善，应去医院就诊。6.对本品过敏者禁用，过敏体质者慎用。7.本品性状发生改变时禁止使用。8.请将本品放在儿童过敏体质者慎用。7.本品性状发生改变时禁止使用。8.请将本品放在儿童不能接触的地方。9.如正在使用其他药品，使用本品前请咨询医师或药师。  尚不明确。"

spans = predict_spans(sample_text)
print("Pred spans:", spans)
for s, e, l in spans:
    print(f"[{l}] {sample_text[s:e]}")